In [31]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import scipy.linalg as la
import time

### Task 21
LU and cholesky decompositions

In [32]:
n = 100

##%%
def frobenius(M):
    return np.sqrt(np.sum(np.power(M,2)))

def compute_lu_error(M, P, L, U):
    P_Inv = la.inv(P)
    L_Inv = la.inv(L)
    U_Inv = la.inv(U)

    M_Inv =  np.dot(np.dot(U_Inv, L_Inv), P_Inv)

    return frobenius(np.dot(M, M_Inv) - np.identity(n))

def compute_cholesky_error(M, L):
    L_Inv = la.inv(L)
    LT_Inv = la.inv(np.transpose(L))

    M_Inv = np.dot(LT_Inv, L_Inv)

    return frobenius(np.dot(M, M_Inv) - np.identity(n))

init tridiagonal matrix A

In [33]:
A = np.zeros((n, n))
for i in range(0, n):
    for j in range(0, n):
        if i == j:
            A[i][j] = 2
        if i == j + 1 or i + 1 == j:
            A[i][j] = -1
            A[i][j] = -1

Hermitian matrix with $a_{ij} = a_{ji}$

In [34]:
B = np.zeros((n, n))
for i in range(0, n):
    for j in range(0, n):
        rand = random.rand(1)
        B[i][j] = rand
        B[j][i] = rand

Compute LU decomposition

In [35]:
start = time.time()
(P, L, U) = la.lu(A)
end = time.time()

print("Time for LU decomposition of A: ", end - start)
print("LU error for A:", compute_lu_error(A, P, L, U))

start = time.time()
(P, L, U) = la.lu(B)
end = time.time()

print("Time for LU decomposition of A: ", end - start)
print("LU error for B:", compute_lu_error(B, P, L, U))

Time for LU decomposition of A:  0.0201570987701416
LU error for A: 6.765944726212899e-13
Time for LU decomposition of A:  0.0013446807861328125
LU error for B: 8.097450162167744e-11


In [36]:
L = la.cholesky(A, lower=True, overwrite_a=False, check_finite=True)
print("Cholesky error for A: " , compute_cholesky_error(A, L))


L = la.cholesky(B, lower=True, overwrite_a=False, check_finite=False)
print("Cholesky error for B: " , compute_cholesky_error(B, L))


Cholesky error for A:  9.3419263010049e-13


LinAlgError: 2-th leading minor of the array is not positive definite

Task 22